In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps
from scipy.sparse import csr_matrix, find
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from inverted_index import get_inverted_index
from search_engine import search

In [2]:
#nltk.download('stopwords')

# Preprocessando

In [3]:
COLUMN_AXIS = 1
CONTENT_COLNAME = 'conteudo'
SUBTITLE_COLNAME = 'subTitulo'
TITLE_COLNAME = 'titulo'
FULL_REPORT_COLNAME = 'noticia'
TOKENS_COLNAME = 'tokens'
TERM_COLNAME = 'term'
REPORT_ID_COLNAME = 'idNoticia'

In [4]:
news = pd.read_csv("../data/estadao_noticias_eleicao.csv", encoding="utf-8")

In [5]:
empty_str = ""
news.titulo.fillna(empty_str, inplace=True)
news.subTitulo.fillna(empty_str, inplace=True)
news.conteudo.fillna(empty_str, inplace=True)

In [6]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
news[FULL_REPORT_COLNAME] = content

In [7]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

In [8]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

In [9]:
tokens_lists = content.apply(lambda text: text.lower().split())

In [10]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [11]:
news = news[[REPORT_ID_COLNAME, FULL_REPORT_COLNAME]]
news[TOKENS_COLNAME] = news[FULL_REPORT_COLNAME].apply(lambda text: tokenizer.tokenize(text.lower()))

# 1 Escreva uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).

In [12]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [13]:
matrix, vocab_to_index = co_occurrence_matrix(tokens)

In [14]:
consultable_matrix = matrix.tocsr()

# 2 Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.

In [24]:
index_to_vocab = {v: k for k, v in vocab_to_index.items()}

In [25]:
three_array_matrix = list(zip(
    matrix.row, matrix.col, matrix.data))

In [26]:
def get_freq(mtuple):
    return mtuple[2]

def get_top3(word, three_array_matrix):
    w_index = vocab_to_index[w1]
    freqs = list(filter(lambda mt: mt[0] == w_index, three_array_matrix))
    max_freq = sorted(freqs, key=get_freq, reverse=True)
    sorted_indexes_by_freq = list(map(lambda mtuple: mtuple[1], max_freq))
    top3_index = sorted_indexes_by_freq[:3]
    top3_words = list(map(lambda i: index_to_vocab[i], top3_index))
    return top3_words

# 3 Expanda a consulta original com os termos retornados no passo 2 acima.

In [48]:
def expand_query(term, top3_words):
    expanded_query = " OR ".join([term] + top3_words)
    return expanded_query

# 4 Faça uma busca disjuntiva (OR) considerando a nova consulta.

Para isso importei meus módulos de [InvertedIndex](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part2/inverted_index.py) e [SearchEngine](https://github.com/italo-batista/information-retrieval/blob/master/01%20Inverted%20Index/part2/search_engine.py). Eles já sabem fazer consultas disjuntivas.

In [50]:
inverted_index = get_inverted_index(news)

# 5 Consultando

Escolha livremente três termos de consulta e responda o seguinte:
- Quais os termos retornados para a expansão de cada consulta?  
- Você acha que esses termos são de fato relacionados com a consulta original? Justifique.  
- Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?  
- A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

In [59]:
query = 'poucos'

top3_words = get_top3(query, three_dim_matrix)
print("Termos retornados para expansão:", ", ".join(top3_words))

query_search_result = search(query, inverted_index)
query_search_result = list(map(str, query_search_result))
print("\nDocumentos retornados para consulta:", ", ".join(query_search_result))


expanded_query = expand_query(w1, top3_words)
expanded_query_search_result = search(expanded_query, inverted_index)
expanded_query_search_result = list(map(str, expanded_query_search_result))
print("\nDocumentos retornados para consulta expandida:", ", ".join(expanded_query_search_result))

Termos retornados para expansão: dias, meses, anos

Documentos retornados para consulta: 2, 13, 28, 42, 48, 60, 109, 139, 161, 162, 173, 200, 222, 235, 342, 353, 402, 403, 411, 423, 433, 435, 444, 518, 549, 567, 603, 615, 659, 680, 682, 693, 760, 783, 804, 836, 855, 865, 889, 915, 949, 996, 1018, 1030, 1033, 1035, 1233, 1235, 1237, 1238, 1241, 1244, 1333, 1385, 1393, 1407, 1437, 1442, 1463, 1503, 1519, 1545, 1589, 1652, 1657, 1658, 1696, 1729, 1837, 1843, 1875, 1891, 1953, 1969, 1970, 1989, 2063, 2110, 2154, 2155, 2183, 2193, 2244, 2274, 2277, 2285, 2294, 2311, 2338, 2339, 2346, 2374, 2385, 2413, 2443, 2454, 2460, 2479, 2480, 2501, 2527, 2574, 2599, 2602, 2635, 2665, 2701, 2707, 2805, 2935, 2986, 2989, 2996, 3009, 3050, 3080, 3088, 3106, 3112, 3163, 3179, 3203, 3260, 3287, 3329, 3338, 3354, 3375, 3396, 3470, 3560, 3649, 3671, 3678, 3743, 3790, 3848, 3886, 3909, 3939, 3942, 3984, 4011, 4034, 4169, 4170, 4187, 4216, 4235, 4244, 4276, 4294, 4319, 4391, 4395, 4399, 4410, 4460, 4479, 4581, 